In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import schedules 
from tensorflow.keras import metrics, losses, callbacks, optimizers 

import coffi3theta
from coffi3theta.core.metrics import compute_jaccard_score, compute_dice_score

import logging
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

sunet_model = coffi3theta.SUNet(
    load_by_name = ('histology', 'Origin'),
    build_model_at_init = True,
    compile_at_init = True
)



# segmentation_path = Path(
#     r'Z:\FA\DATA\NCI\temp\202271003_tiles'
# )
# for imagefile in segmentation_path.iterdir():
#     try:
#         # >>> Segmentation only works on TIFF images <<< #
#         sunet_model.instance_segmentation(segmentation_path, imagefile.name)
#     except:
#         # >>> Skip if the file is not a TIFF image <<< #
#         continue

In [ ]:
import os
import numpy as np
from pathlib import Path

In [ ]:
def generate_overlapping_tiles(numpy_file_path, tile_size, overlap):
    numpy_array = np.load(numpy_file_path)
    
    # get shape of input array
    original_shape = numpy_array.shape
    
    # get dimensions
    height, width, channels = original_shape
    
    # calculate number of tiles in each dimension considering the overlap
    num_tiles_vertical = int(np.ceil((height - overlap) / (tile_size - overlap)))
    num_tiles_horizontal = int(np.ceil((width - overlap) / (tile_size - overlap)))
    
    # create empty list to store the tiles
    tiles = []
    
    for i in range(num_tiles_vertical):
        for j in range(num_tiles_horizontal):
            # calculate starting and ending indices for the current tile
            start_y = i * (tile_size - overlap)
            end_y = min(start_y + tile_size, height)
            start_x = j * (tile_size - overlap)
            end_x = min(start_x + tile_size, width)
            
            # extract current tile from the input array
            tile = numpy_array[start_y:end_y, start_x:end_x, :]
            
            # pad tile if its dimensions are less than tile_size
            padded_tile = np.zeros((tile_size, tile_size, channels), dtype=numpy_array.dtype)
            padded_tile[:tile.shape[0], :tile.shape[1], :] = tile
            
            # append padded tile to the list of tiles
            tiles.append(padded_tile)
    
    # convert list of tiles to a NumPy array
    tiles_array = np.array(tiles)
    
    return tiles_array, original_shape


In [5]:


def process_numpy_file(numpy_file_path):
    # load in the numpy array
    numpy_array = np.load(numpy_file_path)
    
    print("Original array shape:", numpy_array.shape)
    
    # resize array to match desired input shape for models
    desired_shape = (128, 128, 1)
    resized_array = np.resize(numpy_array, desired_shape)
    
    print("Resized array shape:", resized_array.shape)
    
    return resized_array


# instead of reshaping want to have overlap between tiles
# also cmg should contain info of segmentation, not centers 

def process_numpy_files_and_segment(folder_path):
    for numpy_file in os.listdir(folder_path):
        if numpy_file.endswith('.npy'):
            try:
                numpy_file_path = os.path.join(folder_path, numpy_file)
                # load the NumPy array
                tile_array = np.load(numpy_file_path)

                # output folder path
                output_folder_path = Path(folder_path) / (Path(numpy_file).stem + "_segmentation")

                # edit numpy array
                processed_array = process_numpy_file(numpy_file_path)

                # instance segmentation
                # seg_map_path, cmg_file_path = sunet_model.instance_segmentation(processed_array, output_folder_path, Path(numpy_file).stem)
                seg_map_path, cmg_file_path = sunet_model.instance_segmentation(processed_array[np.newaxis, ...], output_folder_path, Path(numpy_file).stem)

                # # debuggin
                # print("Segmentation map saved at:", seg_map_path)
                # print("CMG file saved at:", cmg_file_path)
            except Exception as e:
                # errs
                print(f"Error processing {numpy_file}: {e}")

folder_path = r'C:\Users\axiong\Desktop\SVSTEST\20227003_tiles_numpy'
process_numpy_files_and_segment(folder_path)



Original array shape: (2112, 2112, 3)
Resized array shape: (128, 128, 1)
1/1 [==============================] - 0s 27ms/step
Original array shape: (2176, 2112, 3)
Resized array shape: (128, 128, 1)
1/1 [==============================] - 0s 26ms/step
Original array shape: (2176, 2112, 3)
Resized array shape: (128, 128, 1)
1/1 [==============================] - 0s 31ms/step
Original array shape: (2176, 2112, 3)
Resized array shape: (128, 128, 1)
1/1 [==============================] - 0s 16ms/step
Original array shape: (2176, 2112, 3)
Resized array shape: (128, 128, 1)
1/1 [==============================] - 0s 31ms/step
Original array shape: (2176, 2112, 3)
Resized array shape: (128, 128, 1)
1/1 [==============================] - 0s 24ms/step
Original array shape: (2176, 2112, 3)
Resized array shape: (128, 128, 1)
1/1 [==============================] - 0s 27ms/step
Original array shape: (2176, 2112, 3)
Resized array shape: (128, 128, 1)
1/1 [==============================] - 0s 27ms/step


In [ ]:
input_folder_path = Path(r'C:\Users\axiong\Desktop\SVSTEST\20227001_tiles_numpy')

# Loop over input NumPy arrays
for numpy_file in input_folder_path.iterdir():
    try:
        # Load the NumPy array
        tile_array = np.load(numpy_file)

        # Create the output folder path
        output_folder_path = input_folder_path / (numpy_file.stem + "_segmentation")

        # Perform instance segmentation using the sunet_model 
        seg_map_path, cmg_file_path = sunet_model.instance_segmentation(tile_array, output_folder_path, numpy_file.stem)

        # Print paths to the saved files
        print("Segmentation map saved at:", seg_map_path)
        print("CMG file saved at:", cmg_file_path)
    except Exception as e:
        # handle da errors
        print(f"Error processing {numpy_file}: {e}")








